In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.4 MB/s eta 0:00:00


In [3]:
import os
import json
import random
from datasets import load_dataset
from tqdm import tqdm
from google.colab import drive

drive.mount('/content/drive')

output_dir = '/content/drive/MyDrive/mmml_project'
os.makedirs(output_dir, exist_ok=True)

output_folder = os.path.join(output_dir, "gqa_images")
os.makedirs(output_folder, exist_ok=True)

img_dataset = load_dataset("lmms-lab/GQA", "val_balanced_images")["val"]

img_dict = { str(item["id"]): item["image"] for item in img_dataset }

!mkdir -p /content/gqa/
!wget -O /content/gqa/questions.zip https://downloads.cs.stanford.edu/nlp/data/gqa/questions1.2.zip
!unzip /content/gqa/questions.zip -d /content/gqa/

qa_json_path = '/content/gqa/val_balanced_questions.json'
with open(qa_json_path, 'r') as f:
    qa_data = json.load(f)

if isinstance(qa_data, dict):
    qa_items = list(qa_data.values())
else:
    qa_items = qa_data

random.shuffle(qa_items)

final_data = []
limit = 10000

for qa in tqdm(qa_items, desc="Processing QA pairs"):
    try:
        img_id = str(qa["imageId"])
    except (TypeError, KeyError):
        continue

    if img_id in img_dict:
        image_sample = img_dict[img_id]
        image_file_path = os.path.join(output_folder, f"{img_id}.png")

        if hasattr(image_sample, "save"):
            image_sample.save(image_file_path, format="PNG")
        else:
            image_file_path = image_sample

        merged_entry = {
            "imageId": img_id,
            "image_file": image_file_path,
            "question": qa["question"],
            "answer": qa["answer"],
            "fullAnswer": qa["fullAnswer"]
        }
        final_data.append(merged_entry)
        if len(final_data) >= limit:
            break

metadata_json_path = os.path.join(output_dir, 'mini_gqa.json')
with open(metadata_json_path, 'w') as f:
    json.dump(final_data, f, indent=2)

print(f"Final dataset size: {len(final_data)} datapoints")
print(f"Metadata saved to: {metadata_json_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

val-00000-of-00003.parquet:   0%|          | 0.00/483M [00:00<?, ?B/s]

val-00001-of-00003.parquet:   0%|          | 0.00/483M [00:00<?, ?B/s]

val-00002-of-00003.parquet:   0%|          | 0.00/477M [00:00<?, ?B/s]

Generating val split:   0%|          | 0/10234 [00:00<?, ? examples/s]

--2025-02-21 09:27:32--  https://downloads.cs.stanford.edu/nlp/data/gqa/questions1.2.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1498616372 (1.4G) [application/zip]
Saving to: ‘/content/gqa/questions.zip’

/content/gqa/questi 100%[===================>]   1.40G  4.96MB/s    in 4m 42s  

2025-02-21 09:32:14 (5.08 MB/s) - ‘/content/gqa/questions.zip’ saved [1498616372/1498616372]

Archive:  /content/gqa/questions.zip
  inflating: /content/gqa/challenge_all_questions.json  
  inflating: /content/gqa/challenge_balanced_questions.json  
  inflating: /content/gqa/readme.txt  
  inflating: /content/gqa/submission_all_questions.json  
  inflating: /content/gqa/test_all_questions.json  
  inflating: /content/gqa/test_balanced_questions.json  
  inflating: /content/gqa/testdev_all_questions.json  
  infla

Processing QA pairs:   8%|▊         | 9999/132062 [13:39<2:46:44, 12.20it/s]


Final dataset size: 10000 datapoints
Metadata saved to: /content/drive/MyDrive/mmml_project/mini_gqa.json
